<a href="https://colab.research.google.com/github/Danielasilvac/Livro-POO-com-Python/blob/main/Apostila_14_Collections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exercício: Criando nossa Sequência

In [1]:
from collections.abc import MutableSequence  # Importa a interface MutableSequence para criar uma coleção customizada

class SaldoInsuficienteError(RuntimeError): pass  # Exceção personalizada para saldo insuficiente

class Tributavel:  # Interface para objetos tributáveis
    def get_valor_imposto(self):
        raise NotImplementedError  # Método abstrato: deve ser implementado por subclasses

class Conta:  # Classe base para contas bancárias
    def __init__(self, titular, numero, saldo=0.0, limite=1000.0):
        self._titular = titular
        self._numero = numero
        self._saldo = saldo
        self._limite = limite

    @property
    def saldo(self):
      return self._saldo # Retorna o saldo atual da conta

    def depositar(self, valor):  # Método para depositar valores positivos na conta
        if valor < 0: raise ValueError('Você tentou depositar um valor negativo.')
        self._saldo += valor

    def sacar(self, valor): # Método para sacar valores da conta, com verificação de saldo
        if valor < 0: raise ValueError('Você tentou sacar um valor negativo.')
        if self._saldo < valor: raise SaldoInsuficienteError('Saldo insuficiente.')
        self._saldo -= valor

    def __repr__(self): return f"{self.__class__.__name__}('{self._titular}', '{self._numero}')"
    def __str__(self): return f"{self.__class__.__name__} - Número: {self._numero}, Titular: {self._titular}, Saldo: R$ {self._saldo:.2f}"

class ContaCorrente(Conta, Tributavel):  # Conta corrente com imposto e taxa de saque
    def get_valor_imposto(self):
      return self._saldo * 0.01 # Imposto de 1% sobre o saldo

    def sacar(self, valor): # Método de saque com taxa adicional de R$ 0,10
        if valor < 0: raise ValueError('Você tentou sacar um valor negativo.')
        if self._saldo < valor + 0.10: raise SaldoInsuficienteError('Saldo insuficiente.')
        self._saldo -= (valor + 0.10)

class ContaPoupanca(Conta):  # Conta poupança, sem impostos
    def depositar(self, valor):
        if valor < 0: raise ValueError('Você tentou depositar um valor negativo.')
        self._saldo += valor

class Contas(MutableSequence):  # Classe Contas — coleção personalizada que aceita apenas objetos do tipo Conta
    def __init__(self):
      self._dados = [] # Lista interna para armazenar as contas

    def __len__(self):
      return len(self._dados) # Retorna quantidade de contas

    def __getitem__(self, posicao):
      return self._dados[posicao] # Permite acesso via índice

    def __setitem__(self, posicao, valor):
        if isinstance(valor, Conta):
          self._dados[posicao] = valor # Permite substituir uma conta
        else: raise TypeError("valor atribuído não é uma Conta")

    def __delitem__(self, posicao):
      del self._dados[posicao] # Permite apagar uma conta

    def insert(self, posicao, valor):
        if isinstance(valor, Conta):
          self._dados.insert(posicao, valor) # Permite inserir uma conta
        else: raise TypeError("valor inserido não é uma Conta")

class ManipuladorDeTributaveis:  # Classe utilizada para cálculo de impostos de objetos que implementam Tributavel
    def calcular_impostos(self, lista_tributaveis):
        total = 0
        for item in lista_tributaveis:
            if isinstance(item, Tributavel): total += item.get_valor_imposto()  # Verifica se o item é tributável
        return total

class CaixaEletronico:  # Classe para simular operações bancárias com tratamento de exceções
    def sacar(self, conta, valor):
        try:
            conta.sacar(valor)
            print(f"Saque de R$ {valor:.2f} realizado com sucesso.")
        except ValueError as e: print(f"Erro: {e}")
        except SaldoInsuficienteError as e: print(f"Erro: {e}")

    def depositar(self, conta, valor):
        try:
            conta.depositar(valor)
            print(f"Depósito de R$ {valor:.2f} realizado com sucesso.")
        except ValueError as e: print(f"Erro: {e}")

# Exemplo de uso
if __name__ == '__main__':
    contas_corrente = Contas()  # Cria estrutura de contas

    contas_corrente.append(ContaCorrente('Patrícia', '123-4', 1200.0, 1000.0))
    contas_corrente.append(ContaCorrente('Sérgio', '123-5', 2200.0, 1000.0))
    contas_corrente.append(ContaCorrente('Márcia', '123-6', 1500.0, 1000.0))
    contas_corrente.append(ContaCorrente('Gustavo', '123-7', 5300.0, 1000.0))
    contas_corrente.append(ContaCorrente('Elisa', '123-8', 7800.0, 1000.0))

    print('\nSaldo  -  Imposto')
     # Exibe o saldo e imposto de cada conta
    for c in contas_corrente:
        print(f"{c.saldo:.2f} - {c.get_valor_imposto():.2f}")  # Exibe saldo e imposto

    print("\nSaldo atualizado (após rendimento fictício de 5%)")
    for c in contas_corrente: # Aplica um rendimento de 5% a cada conta e exibe novo saldo
        c.depositar(c.saldo * 0.05)
        print(f"{c.saldo:.2f}")

    manipulador = ManipuladorDeTributaveis()  # Instancia o manipulador de impostos
    total = manipulador.calcular_impostos(contas_corrente)  # Calcula imposto
    print(f"\nTotal de impostos calculados: R$ {total:.2f}")

    print("\nOperações com Caixa Eletrônico")
    caixa = CaixaEletronico()  # Cria o caixa eletrônico
    conta_exemplo = contas_corrente[0] # Seleciona a primeira conta

# Realiza operações de saque e depósito
    caixa.sacar(conta_exemplo, 100)
    caixa.depositar(conta_exemplo, 300)

# Exibe saldo final da conta após as operações
    print("Saldo final:", conta_exemplo.saldo)



Saldo  -  Imposto
1200.00 - 12.00
2200.00 - 22.00
1500.00 - 15.00
5300.00 - 53.00
7800.00 - 78.00

Saldo atualizado (após rendimento fictício de 5%)
1260.00
2310.00
1575.00
5565.00
8190.00

Total de impostos calculados: R$ 189.00

Operações com Caixa Eletrônico
Saque de R$ 100.00 realizado com sucesso.
Depósito de R$ 300.00 realizado com sucesso.
Saldo final: 1459.9
